In [1]:
import pandas as pd
import numpy as np
import os
import networkx as nx
import matplotlib as plt
import matplotlib.pyplot as plt
import random
import math
from itertools import chain #一个库用来提取列表中的列表

In [2]:
#文件读取
network = pd.read_csv(r'J:/IC卡-北京/地铁交通流分配/数据集/subway_network.csv',encoding = 'gbk')
network['Travel_time'] = np.where(network['Source_line'] != network['Target_line'], network['Travel_time'] * 1.86, network['Travel_time'])
stop_num = pd.read_csv(r'J:/IC卡-北京/地铁交通流分配/数据集/stop_num.csv',encoding = 'gbk')
od = pd.read_csv(r'J:/IC卡-北京/IC卡数据_论文/dataset/IC_OD.csv')

In [3]:
od

,card_id,f_line,fst_name,t_line,tst_name,f_timestamp,t_timestamp,Source_num,Target_num
0,1,9,科怡路,95,良乡大学城,1557858540,1557860697,201,344
1,2,2,北京站,6,金安桥,1557823140,1557827133,45,145
2,3,2,西直门,5,北新桥,1557830760,1557832279,54,100
3,4,4,海淀黄庄,1,西单,1557858060,1557859754,83,13
4,5,2,积水潭,15,望京东,1557824280,1557826878,37,374
...,...,...,...,...,...,...,...,...,...
3768159,3768160,90,西局,90,纪家庙,1557864840,1557865776,248,244
3768160,3768161,2,北京站,9,北京西站,1557865800,1557868112,45,166
3768161,3768162,13,知春路,94,沙河高教园,1557864480,1557867082,216,325
3768162,3768163,7,广安门内,93,西红门,1557867660,1557870414,163,64


In [4]:
# 创建空的图
G = nx.Graph()
# 添加节点和边
for _, row in network.iterrows():
    src = row['Source_num']
    dst = row['Target_num']
    time = row['Travel_time']
    src_lon = row['Source_lon']
    src_lat = row['Source_lat']
    dst_lon = row['Target_lon']
    dst_lat = row['Target_lat']   
    
    # 添加节点
    G.add_node(src, pos=(src_lon, src_lat))
    G.add_node(dst, pos=(dst_lon, dst_lat))
    
    # 添加边
    G.add_edge(src, dst, weight=time)

# 设置节点位置
pos = nx.get_node_attributes(G, 'pos')

# 设置边权重
labels = nx.get_edge_attributes(G, 'weight')

# # 绘图
# plt.figure(figsize=(15,15))  # 设置图形的宽度和高度
# nx.draw(G, pos,node_size = 25,node_color = 'orange',font_size = 10,with_labels=True)
# # nx.draw_networkx_edge_labels(G, pos,edge_labels=labels)#edge_labels=labels
# # plt.rcParams['font.family'] = ['MicroSoft YaHei']
# plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
# plt.show()

In [5]:
def select_path(O,D):
    path = pd.DataFrame(columns=['List','Time'])#,'Travel_time'
    shorset_length, shorset_path = nx.single_source_dijkstra(G,source= O, target=D, cutoff=None, weight='weight')
    feasible_paths = nx.all_simple_paths(G,source=O, target=D, cutoff=len(shorset_path) + 3)#cutoff表示中间经过多少点，可以控制经过的点数
    columns = ['num'] #为列重新命名，方便后面的对接
    df = pd.DataFrame(shorset_path, columns=columns) #将最短路的列表转换为表格
    df = pd.merge(df,stop_num,on = 'num',how = 'left')#stop_num是一个csv表格，里面存储了所有编号对应的站点名称
    tra_counts = (df['stop'].value_counts() == 2).sum() #识别出最短路中的换乘站点个数
    
    for j in feasible_paths: #遍历可行路径
        L = list(j) #列表化，方便下面的操作
        if nx.path_weight(G,L,weight='weight') < shorset_length * 1.1:#筛选出阻抗值不超过最短路径10%的路径
            columns = ['num'] #给接下来创建的表格重新命名列名
            df1 = pd.DataFrame(L, columns=columns) #将可行路径列表装换成表格的列数据
            df1 = pd.merge(df1,stop_num,on = 'num',how = 'left') #将站点编号翻译成站点名称
            tra_counts1 = (df1['stop'].value_counts() == 2).sum() #识别出可行路径中的换乘站点个数
            if tra_counts1 < tra_counts +2 or tra_counts1 < 3:#限制换乘站的数量不超过最短路径数量的3个
                t = nx.path_weight(G,j,weight='weight')
                path = path.append({'List': L,'Time': t}, ignore_index=True)
    
    min_time = path['Time'].min()
    total_weight = path['Time'].apply(lambda x: math.e ** (-3*x/min_time)).sum()
    path['P'] = path['Time'].apply(lambda x: (math.e ** (-3*x/min_time))/total_weight)
    path = path.sort_values('P', ascending=True)
    path['Cumu_P'] = path['P'].cumsum()
    
    #生成随0-1之间的随机数，挑选路径
    random_prob = random.uniform(0, 1)
    path_only = pd.DataFrame(columns=path.columns)

    for index, row in path.iterrows():
        if random_prob < row['Cumu_P']:
        # 提取指定行并添加到新表格
            path_only = path_only.append(row, ignore_index=True)
            break
    L= path_only['List'].values
    L = L.tolist()
    L = list(chain.from_iterable(L))
#     L = L[0]
#     return path
#     print(random_prob)
    return L
import warnings
warnings.filterwarnings('ignore')  #忽略pandas发出的无效警告

In [7]:
# od1 = od[0:97] #左闭右开
# od1.reset_index(inplace=True, drop=True) #必须要重置索引，否则后面遍历每一行的时候会出错
od1['Path'] = od1.apply(lambda row: select_path(row['Source_num'],row['Target_num']), axis=1) #遍历每一行，
#将每一行的数据输入 从而计算结果
od1.to_csv('J:/IC卡-北京/地铁交通流分配/数据集/初始路径计算2.0/Path1.csv',encoding = 'gbk', index=False)
od1


In [ ]:
#每次计算9行就输出结果的版本
# chunk_size = 9
# num_chunks = len(od1) // chunk_size + 1

# for i in range(num_chunks):
#     start_idx = i * chunk_size
#     end_idx = (i + 1) * chunk_size
#     chunk_od = od1[start_idx:end_idx]
#     chunk_od['Path'] = chunk_od.apply(lambda row: select_path(row['Source_num'], row['Target_num']), axis=1)
#     chunk_od.to_csv(f'J:/IC卡-北京/地铁交通流分配/数据集/初始路径计算2.0/Path{i+1}.csv', encoding='gbk', index=False)
